In [2]:
# 1. Enable GPU: Runtime -> Change runtime type -> GPU
# 2. Install Libraries
!pip install transformers datasets accelerate

In [3]:
from datasets import load_dataset
from transformers import GPT2Tokenizer

# --- Setup ---
# Load the dataset
dataset_name = "DialogueCharacter/english_soda_unfiltered"
dataset = load_dataset(dataset_name)

# Load the GPT-2 tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# GPT-2 requires the EOS token for padding
tokenizer.pad_token = tokenizer.eos_token

# Define a separation token for dialogue turns.
# GPT-2's EOS token (50256) is often used for this.
SEP_TOKEN = tokenizer.eos_token

# The SODA dataset typically uses 'dialogue' as the main column.
# Let's inspect a sample to confirm the structure and create our formatter.
# print(dataset['train'][0]['dialogue'])
# Assuming each entry in 'dialogue' is a single string representing the whole chat.


README.md:   0%|          | 0.00/455 [00:00<?, ?B/s]

data/train-00000-of-00002-862a034e8a20ad(…):   0%|          | 0.00/253M [00:00<?, ?B/s]

data/train-00001-of-00002-4c68c6afb3db65(…):   0%|          | 0.00/253M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/917016 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:

# # --- Formatting Function ---

# def format_and_tokenize(examples):
#     formatted_texts = []

#     # We assume 'dialogue' column contains the raw text of the conversation.
#     for dialogue_text in examples['dialogue']:
#         # GPT-2 learns by predicting the next token.
#         # We want it to learn to generate the entire dialogue.
#         # Adding SEP_TOKEN at the end is good practice for the model to learn
#         # when a sequence ends.
#         text = f"{dialogue_text.strip()}{SEP_TOKEN}"
#         formatted_texts.append(text)

#     return tokenizer(
#         formatted_texts,
#         truncation=True,
#         max_length=512, # Adjust this based on your average dialogue length and GPU memory
#         padding='max_length'
#     )

# # --- Apply Mapping ---

# # Note: The SODA unfiltered dataset may have other columns, but
# # we only need to remove 'dialogue' and keep the tokenized IDs.
# tokenized_datasets = dataset.map(
#     format_and_tokenize,
#     batched=True,
#     remove_columns=['dialogue']
# )

# # Set the 'input_ids' as 'labels' for Causal Language Modeling
# tokenized_datasets = tokenized_datasets.map(
#     lambda examples: {'labels': examples['input_ids']},
#     batched=True
# )

# print("--- Tokenized Dataset Structure ---")
# print(tokenized_datasets['train'])

In [4]:
# --- Formatting Function (FIXED) ---

def format_and_tokenize(examples):
    formatted_texts = []

    # FIX 1: Iterate over 'instruction' and 'response' columns
    for instruction, response in zip(examples['instruction'], examples['response']):

        # We combine the instruction and response into the dialogue format.
        # Example: <instruction><SEP_TOKEN><response><EOS_TOKEN>
        # This teaches the model to generate the response given the instruction.
        # You can adjust the exact template, but this is a solid starting point:
        text = f"User: {instruction}\nModel: {response}{SEP_TOKEN}"
        formatted_texts.append(text)

    return tokenizer(
        formatted_texts,
        truncation=True,
        max_length=512,
        padding='max_length'
    )

# --- Apply Mapping (FIXED) ---

# FIX 2: Change the list of columns to remove to match the actual dataset columns.
tokenized_datasets = dataset.map(
    format_and_tokenize,
    batched=True,
    remove_columns=['instruction', 'response', 'source'] # Removing the original text columns
)

# Set the 'input_ids' as 'labels' for Causal Language Modeling
tokenized_datasets = tokenized_datasets.map(
    lambda examples: {'labels': examples['input_ids']},
    batched=True
)

print("--- Tokenized Dataset Structure ---")
print(tokenized_datasets['train'])

Map:   0%|          | 0/917016 [00:00<?, ? examples/s]

Map:   0%|          | 0/917016 [00:00<?, ? examples/s]

--- Tokenized Dataset Structure ---
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 917016
})


In [5]:
from transformers import GPT2LMHeadModel, TrainingArguments, Trainer, DataCollatorForLanguageModeling

# --- Load Model ---
model = GPT2LMHeadModel.from_pretrained(model_name)

# --- Define Training Arguments ---
output_dir = "./gpt2_soda_finetuned"

training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    num_train_epochs=3,                   # Recommended for fine-tuning
    per_device_train_batch_size=4,        # Adjust this (4, 8, 16) based on GPU memory
    save_steps=5000,
    save_total_limit=2,
    prediction_loss_only=True,
    logging_steps=500,
    learning_rate=5e-5,                   # Standard learning rate for fine-tuning
    fp16=True,                            # Enable mixed precision training for speed/memory
)

# --- Define Data Collator ---
# DataCollatorForLanguageModeling handles the final batching and ignores the labels
# where the loss shouldn't be computed (like padding tokens).
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False # Crucial: False for Causal Language Modeling (GPT-style)
)

# --- Initialize and Train ---
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
)

print("Starting training...")
trainer.train()

# --- Save Model ---
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)
print(f"Model saved to {output_dir}")

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Starting training...


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


...

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ehabbellkasyalx (ehabbellkasyalx-depi) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


KeyboardInterrupt: 

In [ ]:
from transformers import pipeline

# Load the saved model and tokenizer into a text-generation pipeline
generator = pipeline(
    'text-generation',
    model=output_dir,
    tokenizer=model_name,
    device=0 # Use 0 for the first GPU
)

# Create a prompt that looks like the start of a dialogue from your dataset
prompt = "User: Hey, how was your weekend? Any good plans for next week? Bot: My weekend was great! I finally finished reading the new sci-fi book. Next week I'm planning to"

# Generate the continuation of the dialogue
result = generator(
    prompt,
    max_length=150,
    num_return_sequences=1,
    pad_token_id=tokenizer.eos_token_id
)

print("\n--- Generated Dialogue ---")
print(result[0]['generated_text'])